# List of publications

In [ ]:
import requests
from IPython.display import HTML, display

# === CONFIG ===
ORCID_ID = "0000-0002-0705-7662"   # replace if needed
API_BASE = "https://pub.orcid.org/v3.0"
# ==============

# ORCID session
session = requests.Session()
session.headers.update({"Accept": "application/json"})

def get_all_works(orcid):
    """Return list of put-codes for all works."""
    url = f"{API_BASE}/{orcid}/works"
    r = session.get(url)
    r.raise_for_status()
    works = r.json()

    put_codes = []
    for group in works.get("group", []):
        for ws in group.get("work-summary", []):
            put_codes.append(ws["put-code"])
    return put_codes

def get_work_detail(orcid, put_code):
    """Fetch full metadata for a single work."""
    url = f"{API_BASE}/{orcid}/work/{put_code}"
    r = session.get(url)
    r.raise_for_status()
    return r.json()

def extract_info(work):
    """Extract title, year and DOI from ORCID work."""
    title = (
        work.get("title", {})
            .get("title", {})
            .get("value", "")
    )

    pub_date = work.get("publication-date", {})
    year = pub_date.get("year", {}).get("value")

    doi = None
    for ext in work.get("external-ids", {}).get("external-id", []):
        if ext.get("external-id-type") == "doi":
            doi = ext.get("external-id-value")
            break

    return {
        "title": title.strip(),
        "year": year,
        "doi": doi
    }

def render_publication_list(records):
    """Render publications as a clean HTML bullet list."""
    items = []

    for r in records:
        title = r["title"] or "Untitled"
        year = r["year"]
        doi = r["doi"]

        if doi:
            link = f"https://doi.org/{doi}"
            title_html = f'<a href="{link}" target="_blank">{title}</a>'
        else:
            title_html = title

        year_html = f" ({year})" if year else ""
        items.append(f"<li>{title_html}{year_html}</li>")

    html = f"""
    <ul style="line-height:1.6; margin-left:1.2em;">
        {''.join(items)}
    </ul>
    """
    return HTML(html)

# === RUN ===
put_codes = get_all_works(ORCID_ID)
records = [extract_info(get_work_detail(ORCID_ID, pc)) for pc in put_codes]

# sort newest first
records = sorted(records, key=lambda x: x["year"] or "", reverse=True)

display(render_publication_list(records))